In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras_applications catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from keras.applications.resnet import ResNet50
from PIL import Image
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

In [ ]:
resnet_ = ResNet50(weights='imagenet',classes=1000)

102967424/102967424 [==============================] - 0s 0us/step


In [6]:
dir = '/content/drive/MyDrive/dataset'
test_path = pd.read_csv('/Users/samedi/Downloads/train_dataset_Минприроды/test.csv')
train_path = pd.read_csv('/Users/samedi/Downloads/train_dataset_Минприроды/train.csv')

In [ ]:
len(os.listdir(dir+"/shipun/images"))+len(os.listdir(dir+"/maliy/images"))+len(os.listdir(dir+"/klikun/images"))

9038

In [ ]:
def get_spath(savedir, row_path):
    prepath = row_path['path_img'].replace('images\\','').replace('\\','/')
    prepath = prepath.replace('.jpg','_id'+str(row_path['obj_id'])+'.jpg')
    return os.path.join(savedir, prepath)

In [ ]:
def preprocess(paths, savedir):
    SIZE = (224, 224)
    for i, row_path in paths.iterrows():
        path = os.path.join(dir,row_path['path_img'].replace('\\','/'))
        spath = get_spath(savedir, row_path)
        
        if not os.path.exists(spath):
            img = Image.open(path).convert('RGB')
            img = img.crop((row_path['x1'],row_path['y1'],row_path['x2'],row_path['y2']))
            img = img.resize(SIZE, Image.ANTIALIAS)
            img.save(spath)

        if i%300==299:
            print(i/len(paths.index)*100)

In [ ]:

def extract_batch(paths, dir, batch):
    x = np.ndarray(shape=(batch, 224, 224, 3))
    for i, row_path in paths.iterrows():
        path = get_spath(dir, row_path)
        img = Image.open(path)
        x[i] = np.asarray(img)
    features = resnet_.predict(x, workers=-1)
    df = pd.DataFrame(features)
    df['target'] = paths['type_id']
    return df

def extract(paths, dir, batch=100):
    dfs = []
    N = int(len(paths.index)/batch)
    for i in range(N):
        dfs.append(extract_batch(paths.iloc[i*batch:(i+1)*batch],dir,batch))
        print(str(i/N*100))
    r = len(paths.index)%batch
    if r != 0:
        dfs.append(extract_batch(paths.iloc[len(paths.index)-r:],dir,r))
    return pd.concat(dfs)

In [ ]:
preprocess(train_path,'/content/drive/MyDrive/dataset_crop/train')
preprocess(test_path,'/content/drive/MyDrive/dataset_crop/test')

2.4956180619313915
4.999582672564895
7.503547283198397
10.007511893831902
12.511476504465405
15.015441115098907
17.51940572573241
20.02337033636591
22.527334946999417
25.03129955763292
27.535264168266423
30.039228778899922
32.54319338953343
35.04715800016693
37.551122610800434
40.055087221433936
42.55905183206744
45.06301644270094
47.566981053334445
50.07094566396795
52.57491027460145
55.07887488523495
57.582839495868456
60.08680410650196
62.59076871713547
65.09473332776898
67.59869793840248
70.10266254903598
72.60662715966947
75.11059177030297
77.61455638093648
80.11852099156998
82.62248560220348
85.12645021283699
87.63041482347049
90.13437943410399
92.6383440447375
95.142308655371
97.6462732660045
14.130434782608695
28.30812854442344
42.485822306238184
56.663516068052935
70.84120982986768
85.01890359168242
99.19659735349717


In [ ]:
print(len(os.listdir('/content/drive/MyDrive/dataset_crop/train/klikun')))
print(len(os.listdir('/content/drive/MyDrive/dataset_crop/train/shipun')))
print(len(os.listdir('/content/drive/MyDrive/dataset_crop/train/maliy')))

12341

In [ ]:
len(train_path.index)

11981

In [ ]:
train = extract(train_path,'/content/drive/MyDrive/dataset_crop/train')
test = extract(test_path,'/content/drive/MyDrive/dataset_crop/test')
train.head()

FileNotFoundError: ignored

In [19]:
train = pd.read_csv('/Users/samedi/Downloads/train_dataset_Минприроды/train_ml.csv').drop(columns=['Unnamed: 0'])
test = pd.read_csv('/Users/samedi/Downloads/train_dataset_Минприроды/test_ml.csv').drop(columns=['Unnamed: 0'])

In [23]:
X_train = train.drop(columns=['target'])
X_test = test.drop(columns=['target'])
y_train = pd.DataFrame({'target':train_path['type_id']})
y_test = pd.DataFrame({'target':test_path['type_id']})

In [25]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

c_values = np.logspace(-2, 3, 200)

logit_searcher = LogisticRegressionCV(Cs=c_values, cv=skf, verbose=1, n_jobs=-1)
print(logit_searcher.fit(X_train, y_train))
print('train score',logit_searcher.score(X_train, y_train))
print('valid score',logit_searcher.score(X_test, y_test))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

RUNNING THE L-BFGS-B CODE

           * * *

RUNNING THE L-BFGS-B CODE
RUNNING THE L-BFGS-B CODE


           * * *

           * * *

Machine precision = 2.220D-16
Machine precision = 2.220D-16
Machine precision = 2.220D-16
Machine precision = 2.220D-16
 N =         3003     M =           10

At X0         0 variables are exactly at the bounds
 N =         3003     M =           10
 N =         3003     M =           10
 N =         3003     M =           10

At X0         0 variables are exactly at the bounds

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.05302D+04    |proj g|=  9.02000D+02

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.05302D+04    |proj g|=  9.03000D+02

At iterate    0    f=  1.05291D+04    |proj g|=  9.02333D+02

At iterate    0    f=  1.05302D+04    |proj g|=  9.03000D+02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 3003     15     17      1     0     0   4.983D-02   1.022D+04
  F =   10223.510737114688     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 30

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


...

 This problem is unconstrained.



At iterate   50    f=  9.76308D+03    |proj g|=  1.92880D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 3003     42     53      1     0     0   9.288D-02   9.770D+03
  F =   9770.2176551301600     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         3003     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.76201D+03    |proj g|=  2.48156D-01


 This problem is unconstrained.



At iterate   50    f=  9.76929D+03    |proj g|=  1.01244D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 3003     51     58      1     0     0   3.998D-01   9.769D+03
  F =   9769.2880715148876     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         3003     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.76117D+03    |proj g|=  3.99807D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments 

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


In [24]:
model = CatBoostClassifier(
    iterations=150,
    random_seed=43,
    loss_function='MultiClass'
)

model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [22]:
print(X_train.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11981 entries, 0 to 11980
Columns: 1000 entries, 0 to 999
dtypes: float64(1000)
memory usage: 91.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11981 entries, 0 to 11980
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   target  11981 non-null  int64
dtypes: int64(1)
memory usage: 93.7 KB
None
